In [2]:
import os
import sys

sys.path.append("./")
sys.path.append("../../")

import json

import pandas as pd

from src.config import *

In [12]:
path = "/scratch/salvi/wiki_image_classification/" + MTURK_PATH + "annotated/"
n = 100
seed = 0


df_list = []
for file in filter(
    lambda x: x.startswith(f"{n}_{seed}_")
    and x.endswith(".json")
    and not "combined" in x,
    os.listdir(path),
):
    name = file.split("_")[3].split(".")[0].lower()
    with open(path + file, "r") as f:
        data = pd.DataFrame(json.load(f))
        df_list.append(data[["labels", "other_text"]].add_suffix("_" + name))
df = pd.concat(df_list, axis=1)
df.insert(0, "id", data["id"])

In [13]:
with open(
    "/scratch/salvi/wiki_image_classification/"
    + MTURK_PATH
    + f"{n}_{seed}_sample.json",
    "r",
) as f:
    sample = pd.DataFrame(json.load(f))

In [14]:
df["url"] = sample["url"]

In [15]:
df_labels = df[list(filter(lambda x: x.startswith("labels_"), df.columns))]
df_other = df[list(filter(lambda x: x.startswith("other_text_"), df.columns))]
df["common_labels"] = df_labels.apply(
    lambda x: set.intersection(*[set(l) for l in x.to_list()]), axis=1
)
df["all_labels"] = df_labels.apply(
    lambda x: set.union(*[set(l) for l in x.to_list()]), axis=1
)
df["all_others"] = df_other.apply(lambda x: [el.lower() for el in x if el], axis=1)
df["to_review"] = df.apply(
    lambda x: len(x.common_labels) != len(x.all_labels) or len(x.all_others) > 0, axis=1
)

In [16]:
print(f"Out of {len(df)} images, {sum(df.to_review)} need to be reviewed")

Out of 100 images, 77 need to be reviewed


In [17]:
df.to_json(path + f"{n}_{seed}_sample_combined.json", orient="records")

In [18]:
df

,id,labels_matheus,other_text_matheus,labels_francesco,other_text_francesco,labels_tiziano,other_text_tiziano,url,common_labels,all_labels,all_others,to_review
0,28676777,"[People, History]",,"[People, Art]",,"[People, History, Art]",,https://upload.wikimedia.org/wikipedia/commons...,{People},"{Art, History, People}",[],True
1,13682057,[Architecture],,[Other],Places-Other,[Architecture],,https://upload.wikimedia.org/wikipedia/commons...,{},"{Architecture, Other}",[places-other],True
2,15740891,"[Entertainment, Architecture]",,"[Entertainment, Architecture]",,"[Entertainment, Architecture, Other]",Urban?,https://upload.wikimedia.org/wikipedia/commons...,"{Entertainment, Architecture}","{Entertainment, Architecture, Other}",[urban?],True
3,91008447,"[Technology_Engineering, Transportation]",,"[Technology_Engineering, Transportation]",,"[Technology_Engineering, Transportation]",,https://upload.wikimedia.org/wikipedia/commons...,"{Transportation, Technology_Engineering}","{Transportation, Technology_Engineering}",[],False
4,25942348,"[Technology_Engineering, Transportation]",,"[Technology_Engineering, Transportation]",,"[Technology_Engineering, Transportation]",,https://upload.wikimedia.org/wikipedia/commons...,"{Transportation, Technology_Engineering}","{Transportation, Technology_Engineering}",[],False
...,...,...,...,...,...,...,...,...,...,...,...,...
95,52131824,"[Earth_Environment, Plants, Animals]",,"[Biology, Animals]",,[Animals],,https://upload.wikimedia.org/wikipedia/commons...,{Animals},"{Plants, Biology, Earth_Environment, Animals}",[],True
96,77812701,"[People, History]",,[People],,"[People, History]",,https://upload.wikimedia.org/wikipedia/commons...,{People},"{History, People}",[],True
97,20344861,"[People, Sports, History]",,"[People, Sports]",,"[People, Sports, History]",,https://upload.wikimedia.org/wikipedia/commons...,"{Sports, People}","{Sports, History, People}",[],True
98,9600403,[Other],"Some category in STEM called ""Diagrams"" or som...",[Other],Diagram,[Other],Root of stem,https://upload.wikimedia.org/wikipedia/commons...,{Other},{Other},"[some category in stem called ""diagrams"" or so...",True
